# T06P02 - MdSM

## Enunciado

Para la red de la figura con los datos expresados en valores pu bajo una misma base determine:

1. Intensidad por el neutro del transformador para un fallo fase-tierra (monofásico) en la línea, a 5 km de la barra 2 y con una impedancia de falta a tierra $\mathcal{Z}_f=j0.02\mbox{ pu}$.

2. Tensiones en bornas del generador para la condición de falta anterior.

3. Corrientes de cortocircuito en el punto de fallo, para un fallo fase-fase (bifásico aislado) en la línea, en el mismo punto y con la misma impedancia de contacto que en el caso anterior.

4. Componente homopolar de las tensiones en el punto de fallo para el caso anterior (falta bifásica aislada).

**Sistema eléctrico:**

<img src="./Figs/T06P02_Fig00.png" width=75%>

**NOTA SOBRE EL DESFASE DEL TRANSFORMADOR:**

Obsérvese el desfase introducido por el índice horario (5) del transformador:

<img src="./Figs/Yd5.jpeg" width=65%>
Adapatado de [1].

[1] F. Barrero, Sistemas de Energía Eléctrica, Ed. Paraninfo, 2004, vol 1. pág. 35. ISBN: 978-84-9732-283-6.

## Resolución

In [119]:
# Importar librerías genéricas:
import numpy as np
import cmath as cm
import matplotlib.pyplot as plt

# Formato general de salida de resultados numéricos:
np.set_printoptions(precision=3, suppress=True)

### Datos del problema

In [120]:
# Impedancias a las tres secuencias [homopolar=0, directa=1 e inversa=2] del generador [pu]:
Zg = np.array([0.04j,0.1j,0.08j])
# Impedancias del transformador [pu]:
Zt = np.array([0.12j,0.15j,0.15j])
# Impedancias absolutas de la línea [pu]:
Zl = np.array([0.20j,0.08j,0.08j])
# Longitud de la línea [km]:
l = 20
# Impedancia de cortocircuito de la red [pu]:
Ucc = 1
Scc = 20
Zcc = (Ucc**2/Scc)*1j
Zred = np.array([Zcc,Zcc,Zcc])
# Tensión del generador:
E = 1
# Desfase del transformador [deg]:
Desf = -150 # Yd5.
# Distancia de la falta a la barra 2 [km]:
lf = 5
# Impedancia de falta/contacto [pu]:
Zf = 0.02j
# Impedancia de puesta a tierra del neutro del generador [pu]:
Zn = 0.03j

### Redes de secuencia del sistema

**Red de secuencia homopolar/cero (RSH):**

<img src="./Figs/T06P02_Fig01.png" width=85%>

**Red de secuencia directa (RSD):**

<img src="./Figs/T06P02_Fig02.png" width=85%>

**Red de secuencia inversa (RSI):**

<img src="./Figs/T06P02_Fig03.png" width=85%>

### Cálculo de los equivalentes de Thévenin de las redes de secuencia en el punto de falta

### Red de secuencia homopolar/cero (RSH)

In [121]:
# El equivalente entre el punto de falta y tierra es el resultado del paralelo de dos ramas: izquierda (Rama A0) y derecha (Rama B0).
RamaA0 = Zt[0]+(lf/l)*Zl[0]
RamaB0 = ((l-lf)/l)*Zl[0]+Zred[0]
Z0 = RamaA0*RamaB0/(RamaA0+RamaB0)
print('Z0 = {:.3}'.format(Z0))

Z0 = 0.0919j


### Red de secuencia directa (RSD)

In [122]:
# Ídem que la RSH, teniendo en cuenta que la rama de la izquierda (Rama A1) llega hasta el generador.
RamaA1 = Zg[1]+Zt[1]+(lf/l)*Zl[1]
RamaB1 = ((l-lf)/l)*Zl[1]+Zred[1]
Z1 = RamaA1*RamaB1/(RamaA1+RamaB1)
print('Z1 = {:.3}'.format(Z1))

Z1 = 0.0782j


Obsérvese que la fem de la RSD correspondería con la tensión del equivalente de Thévenin en el punto de análisis. **Nótese en este punto que NO se van a incluir los desfases introducidos por el transformador, los cuáles se tendrán en cuenta sólo a efectos de referirnos a magnitudes del generador**, es decir, a efectos prácticos:

$$\varepsilon^{Th}_1=1\angle 0^\circ.$$

In [123]:
# FEM de la RSD: Tensión del equivalente de Thévenin en el punto de análisis.
E = E
print('E = {:.3f}'.format(E))

E = 1.000


### Red de secuencia inversa (RSI)

In [124]:
# Ídem que la RSD.
RamaA2 = Zg[2]+Zt[2]+(lf/l)*Zl[2]
RamaB2 = ((l-lf)/l)*Zl[2]+Zred[2]
Z2 = RamaA2*RamaB2/(RamaA2+RamaB2)
print('Z2 = {:.3}'.format(Z2))

Z2 = 0.0764j


### Asociación de redes de secuencia (falta monofásica)

Ante falta monofásica a tierra en el punto de falta $F$, las tres redes de secuencia deben asociarse en serie y considerar la impedancia de falta:

<img src="./Figs/T06P02_Fig04.png" width=85%>

Considerando los equivalentes de Thévenin de las tres redes, el esquema queda simplificado:

<img src="./Figs/T06P02_Fig05.png" width=55%>

In [125]:
# Como es falta monofásica, las tres redes se asocian en serie, y por lo tanto el cálculo de las corrientes de las tres redes de secuencia queda:
Z012 = np.array([Z0,Z1,Z2])
I0 = E/(Z012[0]+Z012[1]+Z012[2]+3*Zf)
I1 = I0
I2 = I0
I012 = np.array([I0,I1,I2])
print('I012=',I012)

I012= [0.-3.263j 0.-3.263j 0.-3.263j]


In [126]:
# Por su parte, el cálculo de las tensiones de las tres redes se obtendría mediante:
U0 = -I012[0]*Z012[0]
U1 = E-I012[1]*Z012[1]
U2 = -I012[2]*Z012[2]
U012 = np.array([U0,U1,U2])
print('U012=',U012)

U012= [-0.3  +0.j  0.745+0.j -0.249+0.j]


Ante la falta anterior, la corriente circulante por el neutro del transformador sería igual a la suma vectorial de las corrientes de las tres fases, o lo que es lo mismo, tres veces la corriente homopolar en dicho elemento, que en este caso sería la contribución de la rama de la izquierda (Rama A0) a la corriente homopolar calculada:

$$\mathcal{I}_n^T=\mathcal{I}_R+\mathcal{I}_S+\mathcal{I}_T=3\mathcal{I}_0^T=3\mathcal{I}_0^A.$$

In [127]:
# Obtenemos la corriente homopolar del transformador mediante un divisor de intensidades:
It0 = (RamaB0/(RamaA0+RamaB0))*I012[0]
In = 3*It0
print('La corriente por el neutro del transformador será',round(np.abs(In),3),'pu')

La corriente por el neutro del transformador será 5.292 pu


Por otro lado, la tensión en bornes del generador, es decir, la tensión en la barra 1, se puede obtener mediante la transformación al dominio de fases de las componentes simétricas de la tensión en dicha barra (ante la situación de falta estudiada). Para ello:

1. Determinaremos las componentes simétricas de la tensión en la barra 1, las cuáles obtendremos a partir de las componentes simétricas de las corrientes en el generador y las respectivas caídas de tensión en su impedancia interna:

$$\mathcal{U}^g_1=\mathcal{E}-\mathcal{I}_1^A\mathcal{Z}_1^g.$$

$$\mathcal{U}^g_2=-\mathcal{I}_2^A\mathcal{Z}_2^g.$$

$$\mathcal{U}^g_0=-\mathcal{I}_0^A\mathcal{Z}_0^g.$$

2. Transformación al dominio de fases de las componentes simétricas de la tensión mediante la Matriz de Fortescue ($\left[T\right]$):

$$\left[\mathcal{U}_{RST}^g\right]=\left[T\right]\left[\mathcal{U}_{012}^g\right].$$

**NOTA IMPORTANTE:** Obsérvese que el transformador tiene un índice horario no nulo, lo que significa que introduce un desfase en la RSD, y la opuesta en la RSI. Así, para valores a partir del secundario del transformador, el desfase introducido (en este caso de valor $\alpha=-150^\circ$) debe sumarse en la RSD, mientras que debe restarse en la RSI:

$$\mathcal{U}^{bus1}_1=\mathcal{U}^{bus2}_1e^{-j150^\circ}.$$

$$\mathcal{U}^{bus1}_2=\mathcal{U}^{bus2}_2e^{j150^\circ}.$$

**NOTA 2:** Para saber el sentido en el cual aplicar el desfase, obsérvese en este caso que la tensión en el secundario (bus 1) está atrasada 150 grados ($\phi-150^\circ$) respecto al primario (bus 2).

In [135]:
## Componentes simétricas de la tensión en la barra 1
# Componente homopolar:
Ug0 = 0
# Componente directa:
It1 = (RamaB1/(RamaA1+RamaB1))*I012[1]
Ug1 = E-It1*Zg[1]
# Aplico el desfase del transformador:
Ug1 = Ug1*cm.rect(1,Desf*cm.pi/180)
# Componente inversa:
It2 = (RamaB2/(RamaA2+RamaB2))*I012[2]
Ug2 = -It2*Zg[2]
# Aplico el desfase del transformador:
Ug2 = Ug2*cm.rect(1,-Desf*cm.pi/180)
# Componentes simétricas de la tensión en bornes del generador:
Ug012 = np.array([Ug0,Ug1,Ug2])
print('Ug012=',Ug012)

Ug012= [ 0.   +0.j    -0.722-0.417j -0.121+0.07j ]


De forma alternativa, se podría haber considerado en la resolución que la fem del generador y de la red tienen fase nula, es decir:

$$\varepsilon^g=\varepsilon^{red}=1\angle 0^\circ \mbox{pu},$$
y por lo tanto, considerando el desfase del transformador:

$$\varepsilon^{Th}_1=1\angle 0^\circ \mbox {pu} \cdot 1\angle 150^\circ = 1 \angle 150^\circ \mbox{pu}.$$

Si se considera así, el cálculo de la tensión en bornes del generador ya no requeriría aplicar el desfase del transformador a las componentes de la tensión directa e inversa, pues ya habrían sido calculadas teniendo en cuenta el desfase. La siguiente celda demuestra que el resultado que se obtendría sería el mismo.

In [129]:
# Aplicamos el desfase del transformador a la fem de la fuente equivalente:
E = 1*cm.rect(1,-Desf*cm.pi/180)
print('E = {:.3}'.format(E))

# Recalculamos las corrientes y tensiones en el punto de falta:
I0 = E/(Z012[0]+Z012[1]+Z012[2]+3*Zf)
I1 = I0
I2 = I0
I012 = np.array([I0,I1,I2])

U0 = -I012[0]*Z012[0]
U1 = E-I012[1]*Z012[1]
U2 = -I012[2]*Z012[2]
U012 = np.array([U0,U1,U2])

## Calculamos las componentes simétricas de la tensión en bornes del generador:
# Componente homopolar:
Ug0 = 0
# Componente directa:
It1 = (RamaB1/(RamaA1+RamaB1))*I012[1]
Ug1 = E-It1*Zg[1]
# Componente inversa:
It2 = (RamaB2/(RamaA2+RamaB2))*I012[2]
Ug2 = -It2*Zg[2]
# Componentes simétricas de la tensión en bornes del generador:
Ug012 = np.array([Ug0,Ug1,Ug2])
print('Ug012=',Ug012)

E = (-0.866+0.5j)
Ug012= [ 0.   +0.j    -0.784+0.453j  0.069-0.04j ]


In [130]:
## Transformación de las componentes simétricas a las componentes de fase (cambio de base)
# Matriz de Fortescue-Stovkis
a = cm.rect(1,120*cm.pi/180)
T = np.array([[1,1,1],[1,a**2,a],[1,a,a**2]])
UgRST = np.dot(T,np.transpose(Ug012))
print('Tensión en bornes del generador (coordenadas rectangulares):',UgRST)
# Tensión en bornes del generador en coordenadas polares:
UgRSTpolar = np.zeros([3,2])
for i in range(0,3):
    UgRSTpolar[i] = cm.polar(UgRST[i])
    UgRSTpolar[i][1] = UgRSTpolar[i][1]*180/cm.pi
print('Tensión en bornes del generador (coordenadas polares):',UgRSTpolar)

Tensión en bornes del generador (coordenadas rectangulares): [-0.715+0.413j  0.784+0.533j -0.069-0.945j]
Tensión en bornes del generador (coordenadas polares): [[  0.826 150.   ]
 [  0.948  34.179]
 [  0.948 -94.179]]


### Asociación de redes de secuencia (falta bifásica)

Ante falta bifásica aislada en el punto de falta $F$, las tres redes de secuencia deben asociarse en paralelo y considerar la impedancia de contacto. Como la falta es aislada de tierra, puede observase que la RSH queda igualmente aislada (en circuito abierto). Considerando los equivalentes de Thévenin de las tres redes, el esquema queda simplificado:

<img src="./Figs/T06P02_Fig06.png" width=55%>

Ante la falta bifásica descrita, se deduce que la corriente de la RSH es nula, y las corrientes de las RSD y RSI son iguales y opuestas en signo y se calculan fácilmente como:

$$\mathcal{I}_1^F=-\mathcal{I}_2^F=\displaystyle\frac{\mathcal{E}_1^{Th}}{X_1^{Th}+X_c+X_2^{Th}}.$$

In [131]:
E = 1
I0 = 0
I1 = E/(Z012[1]+Z012[2]+Zf)
I2 = -I1
I012 = np.array([I0,I1,I2])
print('I012=',I012)

I012= [ 0.+0.j     0.-5.729j -0.+5.729j]


De forma análoga, la tensión de la RSH resulta inmediata al estar en circuito abierto y no existir f.e.m. en esta secuencia, mientras que las tensiones de las RSD y RSI se obtienen como:

$$\mathcal{U}^F_1=\mathcal{E}^{Th}_1-\mathcal{I}_1\mathcal{Z}_1.$$

$$\mathcal{U}^F_2=-\mathcal{I}_2\mathcal{Z}_2.$$

In [132]:
U0 = 0
U1 = E-I012[1]*Z012[1]
U2 = -I012[2]*Z012[2]
U012 = np.array([U0,U1,U2])
print('U012=',U012)

U012= [0.   +0.j 0.552+0.j 0.438+0.j]


In [133]:
## Transformación de las componentes simétricas a las componentes de fase (cambio de base)
# Corrientes:
IRST = np.dot(T,np.transpose(I012))
IRSTpolar = np.zeros([3,2])
for i in range(0,3):
    IRSTpolar[i] = cm.polar(IRST[i])
    IRSTpolar[i][1] = IRSTpolar[i][1]*180/cm.pi
print('Corriente en el punto de falta (coordenadas polares):',IRSTpolar)
# Tensiones:
URST = np.dot(T,np.transpose(U012))
URSTpolar = np.zeros([3,2])
for i in range(0,3):
    URSTpolar[i] = cm.polar(URST[i])
    URSTpolar[i][1] = URSTpolar[i][1]*180/cm.pi
print('Tensión en el punto de falta (coordenadas polares):',URSTpolar)

Corriente en el punto de falta (coordenadas polares): [[  0.      0.   ]
 [  9.923 180.   ]
 [  9.923  -0.   ]]
Tensión en el punto de falta (coordenadas polares): [[   0.99     0.   ]
 [   0.505 -168.663]
 [   0.505  168.663]]
